### Evaluation

In [1]:
import json

# Define the path to the JSON file
json_file_path = 'nnUNet_raw_data_base/nnUNet_raw_data/Task006_PancreasUHN/file_mapping.json'

# Open the JSON file and load it into a dictionary
with open(json_file_path, 'r') as f:
    file_mapping = json.load(f)

In [2]:
import pandas as pd

# Set maximum number of rows to display
pd.set_option('display.max_rows', None)

# Set maximum number of columns to display
pd.set_option('display.max_columns', None)

# Set maximum display width
pd.set_option('display.width', 1000)

# Set maximum number of columns to display
pd.set_option('display.max_columns', None)

# Set maximum column width
pd.set_option('display.max_colwidth', None)

df = pd.DataFrame.from_dict(file_mapping, orient='index', columns=['Renamed Path'])

# Rename the index to 'Original Path'
df.index.name = 'Original Path'

# Reset the index to make 'Original Path' a column
df.reset_index(inplace=True)

In [3]:
df.head()

,Original Path,Renamed Path
0,original_data/UHN-MedImg3D-ML-quiz/train/subtype2/quiz_2_416_0000.nii.gz,original_data/Task006_PancreasUHN/imagesTr/case_001
1,original_data/UHN-MedImg3D-ML-quiz/train/subtype2/quiz_2_040_0000.nii.gz,original_data/Task006_PancreasUHN/imagesTr/case_002
2,original_data/UHN-MedImg3D-ML-quiz/train/subtype2/quiz_2_493_0000.nii.gz,original_data/Task006_PancreasUHN/imagesTr/case_003
3,original_data/UHN-MedImg3D-ML-quiz/train/subtype2/quiz_2_321_0000.nii.gz,original_data/Task006_PancreasUHN/imagesTr/case_004
4,original_data/UHN-MedImg3D-ML-quiz/train/subtype2/quiz_2_002_0000.nii.gz,original_data/Task006_PancreasUHN/imagesTr/case_005


In [4]:
df = pd.read_csv('nnUNet_raw_data_base/nnUNet_raw_data/Task006_PancreasUHN/class_mapping.csv')

In [5]:
df_train = df[df['Original Path'].str.contains('train')]
df_test = df[df['Original Path'].str.contains('test')]
df_val = df[df['Original Path'].str.contains('validation')]

In [6]:
df_val['File ID'] = df_val['Original Path'].str.split('/').str[-1].str.replace('_0000.nii.gz', '')

/tmp/ipykernel_1084249/848698125.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_val['File ID'] = df_val['Original Path'].str.split('/').str[-1].str.replace('_0000.nii.gz', '')


In [7]:
val_preds = pd.read_csv('/scratch/alif/nnUNet/original_data/pancreas_validation_preds/predictions.csv')

In [8]:
val_preds['File ID'] = val_preds['File'].str.split('/').str[-1].str.replace('.nii.gz', '')

In [9]:
val_preds.head()

,File,Label,Probs,File ID
0,original_data/pancreas_validation_preds/quiz_1_213.nii.gz,1,"[[0.00011146068572998047, 0.84521484375, 0.154541015625]]",quiz_1_213
1,original_data/pancreas_validation_preds/quiz_0_174.nii.gz,2,"[[0.00951385498046875, 0.22314453125, 0.767578125]]",quiz_0_174
2,original_data/pancreas_validation_preds/quiz_1_158.nii.gz,1,"[[0.0011816024780273438, 0.5205078125, 0.478515625]]",quiz_1_158
3,original_data/pancreas_validation_preds/quiz_0_189.nii.gz,0,"[[0.9169921875, 0.0016565322875976562, 0.08148193359375]]",quiz_0_189
4,original_data/pancreas_validation_preds/quiz_0_184.nii.gz,0,"[[0.6806640625, 0.00827789306640625, 0.31103515625]]",quiz_0_184


In [10]:
import torch
import ast

def extract_max_prob(prob_str):
    # Convert string to tensor
    prob_list = ast.literal_eval(prob_str)[0]
    # Return the maximum probability
    return max(prob_list)

# Apply the function to create a new column
val_preds['Max_Prob'] = val_preds['Probs'].apply(extract_max_prob)

In [11]:
val_preds.head()

,File,Label,Probs,File ID,Max_Prob
0,original_data/pancreas_validation_preds/quiz_1_213.nii.gz,1,"[[0.00011146068572998047, 0.84521484375, 0.154541015625]]",quiz_1_213,0.845215
1,original_data/pancreas_validation_preds/quiz_0_174.nii.gz,2,"[[0.00951385498046875, 0.22314453125, 0.767578125]]",quiz_0_174,0.767578
2,original_data/pancreas_validation_preds/quiz_1_158.nii.gz,1,"[[0.0011816024780273438, 0.5205078125, 0.478515625]]",quiz_1_158,0.520508
3,original_data/pancreas_validation_preds/quiz_0_189.nii.gz,0,"[[0.9169921875, 0.0016565322875976562, 0.08148193359375]]",quiz_0_189,0.916992
4,original_data/pancreas_validation_preds/quiz_0_184.nii.gz,0,"[[0.6806640625, 0.00827789306640625, 0.31103515625]]",quiz_0_184,0.680664


In [12]:
df_val_subset = df_val[['File ID', 'Class Label']]
val_preds_subset = val_preds[['File ID', 'Label', 'Max_Prob']]

merged_df = df_val_subset.merge(val_preds_subset, on='File ID', how='inner')

merged_df['Match'] = merged_df['Class Label'] == merged_df['Label']

total_samples = len(merged_df)
matching_samples = merged_df['Match'].sum()
overlap_percentage = (matching_samples / total_samples) * 100

print(f"Percentage of overlapping class labels: {overlap_percentage:.2f}%")
print(f"Matching samples: {matching_samples}")
print(f"Total samples: {total_samples}")

Percentage of overlapping class labels: 75.00%
Matching samples: 27
Total samples: 36


In [13]:
merged_df.head()

,File ID,Class Label,Label,Max_Prob,Match
0,quiz_2_191,2.0,2,0.843750,True
1,quiz_2_241,2.0,2,0.822266,True
2,quiz_2_084,2.0,1,0.811523,False
3,quiz_2_377,2.0,2,0.532715,True
4,quiz_2_098,2.0,2,0.858398,True


In [14]:
true_labels = merged_df['Class Label'].values

In [15]:
pred_labels = merged_df['Label'].values

In [16]:
pred_probs = merged_df['Max_Prob'].values

In [17]:
from MetricsReloaded.metrics.pairwise_measures import BinaryPairwiseMeasures as BPM
from MetricsReloaded.metrics.prob_pairwise_measures import ProbabilityPairwiseMeasures as PPM
from MetricsReloaded.processes.mixed_measures_processes import MultiLabelPairwiseMeasures as MLPM
from MetricsReloaded.processes.overall_process import ProcessEvaluation as PE
import numpy as np
from matplotlib import pyplot as plt

In [18]:
print('Creation of dictionary for one single comparison case with two metrics')
bpm = BPM(pred_labels, true_labels, measures=['fbeta','mcc'])
dict_seg = bpm.to_dict_meas()
print(dict_seg)

Creation of dictionary for one single comparison case with two metrics
{'fbeta': 0.8656716579104562, 'mcc': 0.46964663}


In [19]:
print("Creation of multi process with error in choice of measure due to absence of probabilistic input")
mlpm = MLPM([pred_labels], [true_labels],[pred_probs],list_values=[1],measures_pcc=['fbeta','mcc'],measures_mt=['auroc'],per_case=True)
df_seg, df_mt = mlpm.per_label_dict()
print(df_seg)

Creation of multi process with error in choice of measure due to absence of probabilistic input
1  is treated label
      fbeta       mcc  label  case
0  0.758621  0.597148      1     0


/scratch/alif/MetricsReloaded/MetricsReloaded/metrics/prob_pairwise_measures.py:190: RuntimeWarning: invalid value encountered in scalar divide
  return self.tp_thr(thresh) / (self.tp_thr(thresh) + self.fp_thr(thresh))


In [20]:
print("Creation of full process")
data = {}
data['pred_class'] = [pred_labels]
data['ref_class'] = [true_labels]
data['pred_prob'] = [pred_probs]
data['list_values'] = [1]
pe = PE(data, 'ImLC',measures_overlap=['fbeta','mcc'],measures_mt=['auroc'],case=True)

Creation of full process
1  is treated label
list fin is  [      fbeta       mcc  label  case
0  0.758621  0.597148      1     0,    auroc  label  case
0    0.5      1     0]
list fin is  [      fbeta       mcc  label  case
0  0.758621  0.597148      1     0,    auroc  label  case
0    0.5      1     0]


/scratch/alif/MetricsReloaded/MetricsReloaded/metrics/prob_pairwise_measures.py:190: RuntimeWarning: invalid value encountered in scalar divide
  return self.tp_thr(thresh) / (self.tp_thr(thresh) + self.fp_thr(thresh))
